In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as tfs
import torchvision.models as models
import numpy as np
import matplotlib.pyplot as plt
from tqdm import trange
import torch.nn.init as init

if torch.cuda.is_available():
  device = 'cuda'
else:
  device='cpu'
print(device)

batch_size = 32
lr = 1e-3


cuda


In [0]:
def Conv2d_(input_channels, output_channels, kernel_size_=3, padding_=1):
  return nn.Conv2d(input_channels, output_channels, kernel_size=kernel_size_, padding=padding_)

class VGG16(nn.Module):

  def __init__(self):
    super(VGG16, self).__init__()
    self.cnn1 = Conv2d_(3, 64)
    self.cnn2 = Conv2d_(64, 128)
    self.cnn3 = Conv2d_(128, 256)
    self.cnn4 = Conv2d_(256, 256)
    self.cnn5 = Conv2d_(256, 512)
    self.cnn6 = Conv2d_(512, 512)
    self.cnn7 = Conv2d_(512, 512)
    self.cnn8 = Conv2d_(512, 512)
    self.fcc = nn.Linear(512, 10)
  
  
  def forward(self, x):
    x = F.max_pool2d(F.relu(self.cnn1(x)), (2, 2))
    x = F.max_pool2d(F.relu(self.cnn2(x)), (2, 2))
    x = F.max_pool2d(F.relu(self.cnn4(F.relu(self.cnn3(x)))), (2, 2))
    x = F.max_pool2d(F.relu(self.cnn6(F.relu(self.cnn5(x)))), (2, 2))
    x = F.max_pool2d(F.relu(self.cnn8(F.relu(self.cnn7(x)))), (2, 2))
    x = x.view(x.size(0), -1)
    x = self.fcc(x)
    return x
  
  def forward_features(self, x):
    f1 = F.max_pool2d(F.relu(self.cnn1(x)), (2, 2))
    f2 = F.max_pool2d(F.relu(self.cnn2(f1)), (2, 2))
    f3 = F.max_pool2d(F.relu(self.cnn4(F.relu(self.cnn3(f2)))), (2, 2))
    f4 = F.max_pool2d(F.relu(self.cnn6(F.relu(self.cnn5(f3)))), (2, 2))
    f5 = F.max_pool2d(F.relu(self.cnn8(F.relu(self.cnn7(f4)))), (2, 2))
    feature = f5.view(f5.size(0), -1)
    x = self.fcc(feature)
    return x, [f1, f2, f3, f4, f5]
  


In [0]:

def _weights_init(m):
  classname = m.__class__.__name__
  #print(classname)
  if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
    init.kaiming_normal_(m.weight)

class LambdaLayer(nn.Module):
  def __init__(self, lambd):
    super(LambdaLayer, self).__init__()
    self.lambd = lambd

  def forward(self, x):
    return self.lambd(x)


class BasicBlock(nn.Module):
  expansion = 1

  def __init__(self, in_planes, planes, stride=1, option='A'):
    super(BasicBlock, self).__init__()
    self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(planes)
    self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn2 = nn.BatchNorm2d(planes)

    self.shortcut = nn.Sequential()
    if stride != 1 or in_planes != planes:
      if option == 'A':
        """
        For CIFAR10 ResNet paper uses option A.
        """
        self.shortcut = LambdaLayer(lambda x:
        F.pad(x[:, :, ::2, ::2], (0, 0, 0, 0, planes//4, planes//4), "constant", 0))
      elif option == 'B':
        self.shortcut = nn.Sequential(
                        nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                        nn.BatchNorm2d(self.expansion * planes) )

  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.bn2(self.conv2(out))
    out += self.shortcut(x)
    out = F.relu(out)
    return out


class ResNet(nn.Module):
  def __init__(self, block, num_blocks, num_classes=10):
    super(ResNet, self).__init__()
    self.in_planes = 16

    self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
    self.bn1 = nn.BatchNorm2d(16)
    self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
    self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
    self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
    self.linear = nn.Linear(64, num_classes)

    self.apply(_weights_init)

  def _make_layer(self, block, planes, num_blocks, stride):
    strides = [stride] + [1]*(num_blocks-1)
    layers = []
    for stride in strides:
      layers.append(block(self.in_planes, planes, stride))
      self.in_planes = planes * block.expansion
    return nn.Sequential(*layers)


  def forward(self, x):
    out = F.relu(self.bn1(self.conv1(x)))
    out = self.layer1(out)
    out = self.layer2(out)
    out = self.layer3(out)
    out = F.avg_pool2d(out, out.size()[3])
    out = out.view(out.size(0), -1)
    out = self.linear(out)
    return out

  def forward_features(self, x):
    x = F.relu(self.bn1(self.conv1(x)))
    f1 = self.layer1(x)
    f2 = self.layer2(f1)
    f3 = self.layer3(f2)
    out = F.avg_pool2d(f3, f3.size()[3])
    out = out.view(out.size(0), -1)
    out = self.linear(out)
    return out, [f1, f2, f3]

def ResNet32():
    return ResNet(BasicBlock, [5, 5, 5])

In [0]:
sourceNet = ResNet32()
targetNet = VGG16()

In [0]:
def getloss(net, x, y, loss_fn):
  y_ = net(x)
  loss = loss_fn(y_, y)
  return loss

def backprop(loss, optimizer):
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

def trainsource(sourcenet, trainloader, valloader, epochs, lr, device):
  sourceNet.to(device)
  optimizer = optim.Adam(sourcenet.parameters(), lr=lr)
  loss_fn = nn.CrossEntropyLoss()
  min_val_loss = 1e9

  for epoch in trange(epochs):
    losses = []
    for step, data in enumerate(trainloader):
      x, y = data
      x = x.to(device)
      y = y.to(device)
      
      y_ = sourcenet(x)
      loss = loss_fn(y_, y)
      backprop(loss, optimizer)
      losses.append(loss.item())
    avg_train_loss = sum(losses) / len(losses)
    losses = []

    for step, data in enumerate(valloader):
      x, y = data
      x = x.to(device)
      y = y.to(device)
      loss = getloss(sourcenet, x, y, loss_fn)
      losses.append(loss.item())
    avg_val_loss = sum(losses) / len(losses)
    
    print(f'epoch {epoch} finished ===========> train loss : {avg_train_loss},    val loss : {avg_val_loss}')
    if min_val_loss >= avg_val_loss:
      print(f"valid loss decreased from {min_val_loss} to {avg_val_loss}")
      min_val_loss = avg_val_loss
      torch.save(sourcenet.state_dict(), f'sourcenet{epoch}.pt')



In [0]:
class FeatureWeight(nn.Module):
  #returns weights btw features : lambda(m, n)

  def __init__(self, source_feature_volume):
    super(FeatureWeight, self).__init__()
    self.fcc = nn.Linear(source_feature_volume, 1)
  
  def forward(self, source_feature):
    source_feature = source_feature.view(source_feature.size(0), -1)
    lambda_weight = F.relu6(self.fcc(source_feature))
    return lambda_weight

class ChannelWeight(nn.Module):
  #returns weights btw channels : w(m, n)

  def __init__(self, source_feature_WH):
    super(ChannelWeight, self).__init__()
    self.fcc = nn.Linear(source_feature_WH, 1)

  def forward(self, source_feature):
    batch_num, channel_num, W, H = source_feature.shape
    source_feature = source_feature.view(batch_num * channel_num, -1)
    output = self.fcc(source_feature)
    output = F.softmax(output.view(batch_num, -1), dim=1)
    return output

class Rnet(nn.Module):
  #point convolution net

  def __init__(self, target_channels, source_channels):
    super(Rnet, self).__init__()
    self.cnn = nn.Conv2d(target_channels, source_channels, kernel_size=1)
  
  def forward(self, target_features):
    output = self.cnn(target_features)
    return output

In [0]:
# source from
# https://github.com/alinlab/L2T-ww/blob/master/train_l2t_ww.py

def weight_feature_matching(S, rT):
  wS = S.size(3)
  wT = rT.size(3)

  if wS==wT:
    delta = S - rT
  else:
    delta = F.interpolate(S, scale_factor=wT/wS, mode='bilinear') - rT
  
  loss = delta.pow(2).mean(3).mean(2)
  return loss

In [8]:
dumy = torch.zeros((32, 3, 32, 32))
outputS, sFeatures = sourceNet.forward_features(dumy)
outputT, tFeatures = targetNet.forward_features(dumy)
print('source features')
for sf in sFeatures:
  print(sf.shape)

print('target features')
for tf in tFeatures:
  print(tf.shape)

source features
torch.Size([32, 16, 32, 32])
torch.Size([32, 32, 16, 16])
torch.Size([32, 64, 8, 8])
target features
torch.Size([32, 64, 16, 16])
torch.Size([32, 128, 8, 8])
torch.Size([32, 256, 4, 4])
torch.Size([32, 512, 2, 2])
torch.Size([32, 512, 1, 1])


In [0]:
sSize = [(16, 32, 32), (32, 16, 16), (64, 8, 8)]
tSize = [(64, 16, 16), (128, 8, 8), (256, 4, 4), (512, 2, 2), (512, 1, 1)]

def init_metanet(sSize, tSize):
  metaNet = []
  for sidx in range(len(sSize)):
    tlist = []
    for tidx in range(len(tSize)):
      sC, sW, sH = sSize[sidx]
      tC, tW, tH = tSize[tidx]

      fNet = FeatureWeight(sC * sW * sH)
      cNet = ChannelWeight(sW * sH)
      rNet = Rnet(tC, sC)
      tlist.append((fNet, cNet, rNet))
    metaNet.append(tlist)
  
  return metaNet

metaNet = init_metanet(sSize, tSize)

In [0]:
def Loss_wfm(tNet, sNet, metaNet, x, y):
  yt, tFeatures = tNet.forward_features(x)
  ys, sFeatures = sNet.forward_features(x)

  loss = 0
  for sidx in range(len(sFeatures)):
    for tidx in range(len(tFeatures)):
      fNet, cNet, rNet = metaNet[sidx][tidx]
      tfeature = tFeatures[tidx]
      sfeature = sFeatures[sidx]

      loss_cij = weight_feature_matching(sfeature, rNet(tfeature))
      w_cmn = cNet(sfeature)
      loss_wfm_mn = torch.sum(w_cmn * loss_cij, dim=1)

      lambda_mn = fNet(sfeature)
      loss += torch.sum(lambda_mn * loss_wfm_mn)
  return loss

def Loss_ori(tNet, x, y):
  loss_fn = nn.CrossEntropyLoss()
  y_ = tNet(x)
  loss = loss_fn(y_, y)
  return loss

def Loss_tot(tNet, sNet, metaNet, beta, x, y):
  return Loss_ori(tNet, x, y) + beta * Loss_wfm(tNet, sNet, metaNet, x, y) 

In [0]:
def backprop_retain(loss, optimizer):
  optimizer.zero_grad()
  loss.backward(retain_graph=True)
  optimizer.step()

def backprop_meta(loss, meta_optimizer, slen, tlen):
  for sidx in range(slen):
    for tidx in range(tlen):
      foptim, coptim, roptim = meta_optimizer[sidx][tidx]
      backprop_retain(loss.clone(), foptim)
      backprop_retain(loss.clone(), coptim)
      backprop_retain(loss.clone(), roptim)

def metaNet_init_optim(metaNet, lr, device):
  mOptim = []
  for sidx in range(len(metaNet)):
    fcrOptim = []
    for tidx in range(len(metaNet[sidx])):
      fNet, cNet, rNet = metaNet[sidx][tidx]
      fNet.to(device)
      cNet.to(device)
      rNet.to(device)

      foptim = optim.Adam(fNet.parameters(), lr=lr)
      coptim = optim.Adam(cNet.parameters(), lr=lr)
      roptim = optim.Adam(rNet.parameters(), lr=lr)
      fcrOptim.append((foptim, coptim, roptim))

    mOptim.append(fcrOptim)
  return mOptim

def metaNet_prepare_train(metaNet):
  for sidx in range(len(metaNet)):
    for tidx in range(len(metaNet[sidx])):
      fNet, cNet, rNet = metaNet[sidx][tidx]
      fNet.train()
      cNet.train()
      rNet.train()

def metaNet_prepare_test(metaNet):
  for sidx in range(len(metaNet)):
    for tidx in range(len(metaNet[sidx])):
      fNet, cNet, rNet = metaNet[sidx][tidx]
      fNet.eval()
      cNet.eval()
      rNet.eval()

def TrainWWT(tNet, sNet, metaNet, trainloader, valloader, epochs, lr, T, beta, device):
  tNet.to(device)
  sNet.to(device)
  tOptim = optim.Adam(tNet.parameters(), lr=lr)
  mOptim = metaNet_init_optim(metaNet, lr, device)

  for epoch in trange(epochs):
    tot_losses = []
    ori_losses = []
    wfm_losses = []

    sNet.eval()
    tNet.train()
    metaNet_prepare_train(metaNet)

    for step, data in enumerate(trainloader):
      x, y = data
      x = x.to(device)
      y = y.to(device)

      tot_loss = Loss_tot(tNet, sNet, metaNet, beta, x, y)
      backprop(tot_loss, tOptim)
      tot_losses.append(tot_loss.item())

      for t in range(T):
        wfm_loss = Loss_wfm(tNet, sNet, metaNet, x, y)
        backprop(wfm_loss, tOptim)
        wfm_losses.append(wfm_loss.item())
      
      ori_loss = Loss_ori(tNet, x, y)
      backprop(ori_loss, tOptim)
      
      ori_loss = Loss_ori(tNet, x, y)
      backprop_meta(ori_loss, mOptim, len(metaNet), len(metaNet[0]))
      ori_losses.append(ori_loss.item())
    
    avg_tot_loss = sum(tot_losses) / len(tot_losses)
    avg_ori_loss = sum(ori_losses) / len(ori_losses)
    avg_wfm_loss = sum(wfm_losses) / len(wfm_losses)
    print(f'epoch : {epoch}  train step =======>  avg_tot_loss : {avg_tot_loss}   avg_ori_loss : {avg_ori_loss},  avg_wfm_loss : {avg_wfm_loss}')

    metaNet_prepare_test(metaNet)
    sNet.eval()
    tNet.eval()

    tot_losses = []
    ori_losses = []
    wfm_losses = []
    for step, data in enumerate(valloader):
      x, y = data
      x = x.to(device)
      y = y.to(device)

      tot_loss = loss_tot(tNet, sNet, metaNet, beta, x, y)
      wfm_loss = loss_wfm(tNet, sNet, metaNet, x, y)
      ori_loss = loss_ori(tNet, x, y)
      tot_losses.append(tot_loss.item())
      wfm_losses.append(wfm_loss.item())
      ori_losses.append(ori_loss.item())

    avg_tot_loss = sum(tot_losses) / len(tot_losses)
    avg_ori_loss = sum(ori_losses) / len(ori_losses)
    avg_wfm_loss = sum(wfm_losses) / len(wfm_losses)
    print(f'epoch : {epoch}  valid step =======>  avg_tot_loss : {avg_tot_loss}   avg_ori_loss : {avg_ori_loss},  avg_wfm_loss : {avg_wfm_loss}')

In [0]:
#load data and train source net....
#don't have access to tiny imagenet... I'm sorry


In [23]:
transform = tfs.Compose([tfs.ToTensor(), tfs.Normalize(0.5, 0.5)])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
valset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
valloader = DataLoader(valset, batch_size=32, shuffle=True)

TrainWWT(targetNet, sourceNet, metaNet, trainloader, valloader, 2, 1e-3, 2, 1e-7, device)

Files already downloaded and verified
Files already downloaded and verified



  0%|          | 0/2 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2973: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "


KeyboardInterrupt: ignored